In [10]:
import pandas as pd
import mylib

In [11]:
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 1000)

In [12]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train["target"] = mylib.target_label(train)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
 5   target       34250 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.6+ MB


th=.25, f1=.586
th=.30, f1=.586
th=.35, f1=.587
th=.40, f1=.583

In [13]:
%%time
train["phash_matches"] = mylib.phash_matches(train, threshold=0.35)

CPU times: user 28.8 s, sys: 14.5 s, total: 43.3 s
Wall time: 43.4 s


In [14]:
posting_ids = train["posting_id"].tolist()
sentences = train["title"].tolist()

In [15]:
%%time
# stsb-distilbert-base
st_name = "paraphrase-distilroberta-base-v1"
train["sbert_matches"] = mylib.sbert_matches(
    model_path=f"pretrained/sentence-transformers/{st_name}",
    sentences=sentences,
    posting_ids=posting_ids,
    threshold=0.4
)

CPU times: user 31min 39s, sys: 1min 14s, total: 32min 53s
Wall time: 6min 19s


In [16]:
fs = ["phash_matches", "sbert_matches"]
train["matches"] = train.apply(mylib.combine_as_list(fs), axis=1)
train["f1"] = train.apply(mylib.metric_per_row("matches"), axis=1)
print(f"Combined score={train.f1.mean():.3f}")

Combined score=0.613


In [17]:
cols = ["f1", "target", "matches"] + fs
train[cols].head(30)

,f1,target,matches,phash_matches,sbert_matches
0,0.666667,"[train_129225211, train_2278313361]",[train_129225211],[],[]
1,0.666667,"[train_3386243561, train_3423213080]",[train_3386243561],[],[]
2,0.666667,"[train_2288590299, train_3803689425]",[train_2288590299],[],[]
3,0.500000,"[train_2406599165, train_3342059966]","[train_2406599165, train_1744956981]",[],[train_1744956981]
4,0.666667,"[train_3369186413, train_921438619]",[train_3369186413],[],[]
5,0.800000,"[train_2464356923, train_2753295474, train_305884580]","[train_2753295474, train_2464356923]",[train_2753295474],[]
6,0.250000,"[train_1802986387, train_1396161074, train_713073906, train_1275191373, train_2490201622, train_2411544001, train_1859060005]",[train_1802986387],[],[]
7,0.666667,"[train_1806152124, train_3227306976]",[train_1806152124],[],[]
8,0.500000,"[train_86570404, train_2837452969, train_77364776]",[train_86570404],[],[]
9,0.666667,"[train_831680791, train_3031035861]",[train_831680791],[],[]


In [18]:
train.sort_values("f1", ascending=True, inplace=True, ignore_index=True)
train[cols].head(20)

,f1,target,matches,phash_matches,sbert_matches
0,0.034483,"[train_1251926547, train_576940044, train_2821917682, train_3942479788, train_2262165918, train_2713325894, train_4024729812, train_3500331962, train_1833257672, train_239849978, train_2416568905, train_1605540675, train_2411860849, train_1533892478, train_2921972011, train_3901859980, train_4084569444, train_1359948092, train_2361997442, train_2836494610, train_2767216329, train_1130274962, train_1642454560, train_1625513302, train_2997354311, train_2811016096, train_1518569647, train_4128551344, train_2475719623, train_3178048969, train_389026721, train_3609122916, train_155628598, train_1025570379, train_876376110, train_319894214, train_2573060829, train_745074374, train_3307385969, train_378911549, train_422235145, train_2980795248, train_2419943180, train_493684647, train_2343771203, train_4071624010, train_2032306771, train_4153223724, train_221812587, train_1796048911, train_3730694224]","[train_2623930891, train_1543918412, train_2638290284, train_1636915192, train_3434353278, train_2419943180, train_3494783391]","[train_1636915192, train_1543918412, train_2638290284, train_3434353278, train_2623930891, train_3494783391]",[]
1,0.034483,"[train_1830018286, train_3288013522, train_2902950555, train_293840683, train_1074412229, train_4198005107, train_2347322245, train_4243340733, train_4100605562, train_1183683723, train_3920899944, train_893086606, train_2674186772, train_479485479, train_1168615490, train_152227271, train_3893707485, train_2961324318, train_678760988, train_2191228989, train_305244047, train_3181232330, train_1964782712, train_2421378296, train_2804316316, train_3538863894, train_3680859397, train_1434134137, train_2139641946, train_1914688126, train_4168914734, train_2372985021, train_1594934821, train_961843295, train_3410184654, train_269585399, train_2089790010, train_2911017787, train_3083301272, train_1161773582, train_3386323765, train_2859820744, train_3736379506, train_512268036, train_2606126135, train_497353894]","[train_122952880, train_2456589775, train_2681444479, train_4189824924, train_2904843101, train_1714461223, train_3096617588, train_3042172278, train_530130322, train_2655044356, train_4198005107, train_754954579]","[train_2655044356, train_122952880, train_3042172278, train_3096617588, train_4189824924, train_2681444479, train_2904843101, train_1714461223, train_530130322, train_2456589775, train_754954579]",[]
2,0.036364,"[train_653606694, train_3694367120, train_1676438253, train_723113580, train_718395879, train_3839423258, train_2983368523, train_527397407, train_2599912967, train_1525971777]","[train_415825584, train_62116380, train_2361997442, train_1748484263, train_2080010196, train_2930414767, train_1248209230, train_4153223724, train_1604307785, train_422235145, train_1553061748, train_639259659, train_2717736891, train_3942479788, train_1676438253, train_3079456842, train_2095256261, train_2047191940, train_3500331962, train_209942350, train_1029471304, train_959879012, train_2573060829, train_2565641823, train_3123468195, train_2051118753, train_745074374, train_2343771203, train_247715515, train_1506002495, train_2740619290, train_563561141, train_1249841799, train_2072099738, train_1419271811, train_1073270861, train_3857398510, train_1461409280, train_3417411762, train_4128551344, train_1258915345, train_3011723573, train_2044493665, train_2446942306, train_288413860]","[train_3079456842, train_3011723573, train_1248209230, train_1461409280, train_3857398510, train_2072099738, train_3942479788, train_639259659, train_288413860, train_2080010196, train_209942350, train_1029471304, train_1073270861, train_1506002495, train_2446942306, train_2361997442, train_563561141, train_2044493665, train_2095256261, train_1604307785, train_2051118753, train_415825584, train_3123468195, train_1419271811, train_422235145, train_2047191940, train_959879012, train_3417411762, train_4128551344, train_247715515, train_12589153